In [4]:
max1demand=10   #use 1 more than actual demand
max2demand=10    #use 1 more than actual demand
max3demand=4    #use 1 more than actual demand
sparti=[]  #start with null
spartj=[]  #start with null
spartk=[]  #start with null
spartl=[]  #start with null
spartm=[]
spartn=[]
sidx=[]    #start with null
s=1        #start number for loop
for i in 1:4          #Create an index to call
    for j in 1:4
        for k in 1:max1demand
            for l in 1:max2demand
                for m in 1:2
                    for n in 1:2
          #statei(i+1) = s
            sparti = push!(sparti,i)  #for i in ijkl add to sparti definition
            spartj = push!(spartj,j) #for j in ijkl add to spartj definition
            spartk = push!(spartk,k) #for k in ijkl add to spartk definition
            spartl = push!(spartl,l) #for l in ijkl add to spartl definition
            spartm = push!(spartm,m) #for m in ijkl add to spartl definition
            spartn = push!(spartn,n) #for n in ijkl add to spartl definition            
            sidx= push!(sidx,s)      #create index in sidx
            s=s+1
                    end 
                end
            end
        end
    end
end

#sparti
#spartj
#spartk
#[sparti,spartj,spartk]
#sidx

#idxactive=  (sparti.>=4) .& (spartj+spartk+spartl.==3) ##(Wrapped into regional indices)
idxidle= (sparti.<3) .& (spartj.<3) .& (spartk+spartl .>2)   #index all ijkl where idling cannot occur
idxidle2= (sparti.>=3) .& (spartj.<3) .& (spartk+spartl .>3) #index where more than 2 calls exist but ambulance 2 is not busy
idxidle1= (spartj.>=3) .& (sparti.<3) .& (spartk+spartl .>3) #index where more than 2 calls exist but ambulance 1 is not busy
idxregion1= ((sparti.==3) .| (spartj.==3)) .& (spartk.==1)      #index all ijkl where region 1 is active but there are no calls in region 1 to service
idxregion2= ((sparti.==4) .| (spartj.==4)) .& (spartl.==1)      #index all ijkl where region 2 is active but there are no calls in region 2 to service
idxregion11= ((sparti.==3) .& (spartj.==3)) .& (spartk.==2)      #index all ijkl where region 1 is active with both ambulances but there is only 1 call in region 1 to service
idxregion21= ((sparti.==4) .& (spartj.==4)) .& (spartl.==2)      #index all ijkl where region 2 is active with both ambulances but there is only 1 call in region 2 to service


idxDEL=idxidle .| idxregion1 .| idxregion2 .| idxidle1  .| idxidle2 .| idxregion11 .| idxregion21 #index of the previous 3 indices

deleteat!(sparti, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartj, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartk, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartl, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartm, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartn, BitArray(idxDEL)) #delete idxDEL  
sidx= [1:length(sparti);]       #number of states

1336-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
   11
   12
   13
    ⋮
 1325
 1326
 1327
 1328
 1329
 1330
 1331
 1332
 1333
 1334
 1335
 1336

In [286]:
μ1idx=zeros(length(sidx))

46-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [5]:
#Add Arrival Code
λst=.5  #Define overall Arrival Rate for every zone
#λ1=.5  #Define Arrival Rate for Zone1
#λ2=.5  #Define Arrival Rate for Zone2                
μMatr=[1.5 1; 
        1 1.5]
# μ11=1  #Define Service Rate from Zone1 to Zone1
# μ12=.7 #Define Service Rate from Zone1 to Zone2
# μ21=.7 #Define Service Rate from Zone2 to Zone1
# μ22=1  #Define Service Rate from Zone2 to Zone2
matrix= zeros(length(sidx),length(sidx))
μ1idx=zeros(length(sidx))
μ2idx=zeros(length(sidx))
λidx=zeros(length(sidx))
using LinearAlgebra


In [6]:
 for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    RegionFrom1= spartm[s]
    RegionFrom2= spartn[s]
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nextRegionFrom1=0
    nextRegionFrom2=0
    nexts=0
    λ= 0    
 #   Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2        
        if Zoneflag1 == 1 
            if Zoneflag2>1
             nextZoneflag1=3
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=1
             nextRegionFrom2=RegionFrom2
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ       
             λidx[s]=1 
            else 
             λ=λst/2
             nextRegionFrom1=1
             nextRegionFrom2=1
             idx= (sparti.==3) .& (spartj.==1) .& (spartk.==nextCall1) .&(spartl.==nextCall2).& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             idx= (sparti.==1) .& (spartj.==3) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
             λidx[s]=1 
            end
        elseif Zoneflag1 == 2 
            if Zoneflag2>2
             nextZoneflag1=3
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=2
             nextRegionFrom2=RegionFrom2
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
             λidx[s]=1    
            elseif Zoneflag2==1
             nextZoneflag1=Zoneflag1
             nextZoneflag2=3
             nextRegionFrom1=RegionFrom1
             nextRegionFrom2=1 
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ
             λidx[s]=1 
            else 
             λ=λst/2   
             nextRegionFrom1=2
             nextRegionFrom2=2                 
             idx= (sparti.==3) .& (spartj.==2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             idx= (sparti.==2) .& (spartj.==3) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             λidx[s]=1 
            end            
        elseif Zoneflag1>2 && Zoneflag2<=2
            nextZoneflag1 = Zoneflag1  
            nextZoneflag2 = 3 
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=Zoneflag2
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ        
            λidx[s]=1 
        else Zoneflag1>2 && Zoneflag2>2
            nextZoneflag1 = Zoneflag1
            nextZoneflag2 = Zoneflag2
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=RegionFrom2
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ  
            λidx[s]=1 
        end        
    end
end  

In [8]:
sum(matrix)

0.0

In [12]:
  for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    RegionFrom1= spartm[s]
    RegionFrom2= spartn[s]    
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nextRegionFrom1=0
    nextRegionFrom2=0    
    nexts=0
    λ= 0  
  #   Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1               
        if Zoneflag1 == 2 
            if Zoneflag2!=2
             nextZoneflag1=4
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=2
             nextRegionFrom2=RegionFrom2
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ
                λidx[s]=1 
            else 
             λ=λst/2   
             nextRegionFrom1=2
             nextRegionFrom2=2            
             idx= (sparti.==4) .& (spartj.==2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             idx= (sparti.==2) .& (spartj.==4) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ
                λidx[s]=1 
            end
        elseif Zoneflag1 == 1
            if Zoneflag2>2
             nextZoneflag1=4
             nextZoneflag2=Zoneflag2
             nextRegionFrom1=1
             nextRegionFrom2=RegionFrom2                            
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
                λidx[s]=1 
            elseif Zoneflag2==2                
             nextZoneflag1=Zoneflag1
             nextZoneflag2=4
             nextRegionFrom1=RegionFrom1
             nextRegionFrom2=2                  
             λ=λst
             idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ    
                λidx[s]=1 
            else 
             λ=λst/2   
             nextRegionFrom1=1
             nextRegionFrom2=1                  
             idx= (sparti.==4) .& (spartj.==1) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].= matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ 
             idx= (sparti.==1) .& (spartj.==4) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
             nexts= sidx[idx] 
             matrix[nexts,s].=matrix[nexts,s].+λ
             matrix[s,s]= matrix[s,s]-λ   
                λidx[s]=1 
            end            
        elseif Zoneflag1>2 && Zoneflag2<=2
            nextZoneflag1 = Zoneflag1  
            nextZoneflag2 = 4
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=Zoneflag2   
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ 
            λidx[s]=1 
        
        else Zoneflag1>2 && Zoneflag2>2
            nextZoneflag1 = Zoneflag1
            nextZoneflag2 = Zoneflag2
            nextRegionFrom1=RegionFrom1
            nextRegionFrom2=RegionFrom2
            λ=λst
            idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2) .& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
            nexts= sidx[idx] 
            matrix[nexts,s].= matrix[nexts,s].+λ
            matrix[s,s]= matrix[s,s]-λ
            λidx[s]=1 
        end
    end
end
sum(matrix)

0.0

In [290]:
for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    RegionFrom1= spartm[s]
    RegionFrom2= spartn[s]    
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nextRegionFrom1= 0
    nextRegionFrom2= 0    
    nexts=0
    RegionService= 0         
     #Service from Region to Region    
     if Zoneflag1 == 3
        if Zoneflag2 == 4
            if Call2 > Call1
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 4
                nextZoneflag2= Zoneflag2
                nextRegionFrom1=1
                nextRegionFrom2=2
                nextCall1= Call1-1
                nextCall2= Call2
            else
                if Call1==2
                    RegionService = μMatr[1,RegionFrom1]
                    nextZoneflag1= 1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1-1
                    nextCall2= Call2
                    nextRegionFrom1=1
                    nextRegionFrom2=2
                else 
                    RegionService = μMatr[1,RegionFrom1]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1-1
                    nextCall2= Call2   
                    nextRegionFrom1=1
                    nextRegionFrom2=2                    
                end 
            end           
        elseif Zoneflag2==3
            if Call2>=Call1-1
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 4
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2
                nextRegionFrom1=1
                nextRegionFrom2=1                   
            elseif Call1>3
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=1
                nextRegionFrom2=1  
            else
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=1
                nextRegionFrom2=1  
            end 
        else 
                RegionService = μMatr[1,RegionFrom1]
                nextZoneflag1= 1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                nextRegionFrom1=1
                nextRegionFrom2=1  
        end      
    elseif Zoneflag1==4  
          if Zoneflag2 == 3
            if Call1 > Call2
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 3
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1
                nextRegionFrom1=2
                nextRegionFrom2=1                  
            else
                if Call2==2
                    RegionService = μMatr[2,RegionFrom1]
                    nextZoneflag1= 2
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1
                    nextCall2= Call2-1   
                    nextRegionFrom1=2
                    nextRegionFrom2=1 
                else 
                    RegionService = μMatr[2,RegionFrom1]
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1
                    nextCall2= Call2-1      
                    nextRegionFrom1=2
                    nextRegionFrom2=1 
                end 
            end            
        elseif Zoneflag2==4
            if Call1>=Call2-1
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 3
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1
                nextRegionFrom1=2
                nextRegionFrom2=2 
            elseif Call1>1
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1  
                nextRegionFrom1=2
                nextRegionFrom2=2 
            else 
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 2
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1   
                nextRegionFrom1=2
                nextRegionFrom2=2 
            end 
        else 
                RegionService = μMatr[2,RegionFrom1]
                nextZoneflag1= 2
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1     
                nextRegionFrom1=2
                nextRegionFrom2=2 
        end         
    else
    end        
         idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2).& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
         nexts= sidx[idx] 
         matrix[nexts,s]= matrix[nexts,s] .+ RegionService
         matrix[s,s]= matrix[s,s].- RegionService     
end
matrix


46-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 2.0
 2.0
 ⋮
 1.0
 2.0
 2.0
 1.0
 1.0
 1.0
 2.0
 1.0
 2.0
 2.0
 2.0
 2.0

In [294]:
for s in 1:length(sidx)
    Zoneflag1 = sparti[s]
    Zoneflag2= spartj[s]
    Call1 = spartk[s]
    Call2 = spartl[s]
    nextZoneflag1 = 0
    nextZoneflag2 = 0
    nextCall1=0
    nextCall2=0
    nexts=0
    RegionService= 0   
     if Zoneflag2 == 4
        if Zoneflag1 == 3
            if Call1 > Call2
                RegionService = μ2Diff
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 3
                nextCall1= Call1
                nextCall2= Call2-1
                μ2idx[s]=2
            else
                if Call2==2
                    RegionService = μ2Same
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= 2
                    nextCall1= Call1
                    nextCall2= Call2 -1    
                    μ2idx[s]=1
                else 
                    RegionService = μ2Same
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1
                    nextCall2= Call2-1     
                    μ2idx[s]=1
                end 
            end           
        elseif Zoneflag1 == 4
            if Call1>=Call2-1
                RegionService = μ2Diff
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 3
                nextCall1= Call1
                nextCall2= Call2-1
                μ2idx[s]=2
            elseif Call1>1
                RegionService = μ2Same
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1
                nextCall2= Call2-1 
                μ2idx[s]=1
            else                
                RegionService = μ2Same
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 2
                nextCall1= Call1
                nextCall2= Call2-1 
                μ2idx[s]=1
            end 
        else 
                RegionService = μ2Same
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 2
                nextCall1= Call1
                nextCall2= Call2-1 
                μ2idx[s]=1
        end        
    elseif Zoneflag2 == 3
        if Zoneflag1 == 4
            if Call2 > Call1
                RegionService = μ2Diff
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 4
                nextCall1= Call1-1
                nextCall2= Call2
                μ2idx[s]=2
            else
                if Call1==2
                    RegionService = μ2Same
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= 1
                    nextCall1= Call1 -1
                    nextCall2= Call2    
                    μ2idx[s]=1
                else 
                    RegionService = μ2Same
                    nextZoneflag1= Zoneflag1
                    nextZoneflag2= Zoneflag2
                    nextCall1= Call1-1
                    nextCall2= Call2 
                    μ2idx[s]=1
                end 
            end           
        elseif Zoneflag1==3
            if Call2>=Call1-1
                RegionService = μ2Diff
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 4
                nextCall1= Call1-1
                nextCall2= Call2
                μ2idx[s]=2
            elseif Call2>1
                RegionService = μ2Same
                nextZoneflag1= Zoneflag1
                nextZoneflag2= Zoneflag2
                nextCall1= Call1-1
                nextCall2= Call2  
                μ2idx[s]=1
            else                    
                RegionService = μ2Same
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 1
                nextCall1= Call1-1 
                nextCall2= Call2    
                μ2idx[s]=1
            end 
        else 
                RegionService = μ2Same
                nextZoneflag1= Zoneflag1
                nextZoneflag2= 1
                nextCall1= Call1-1
                nextCall2= Call2     
                μ2idx[s]=1
        end          
    end
 idx= (sparti.==nextZoneflag1) .& (spartj.==nextZoneflag2) .& (spartk.==nextCall1) .&(spartl.==nextCall2).& (spartm.==nextRegionFrom1) .&(spartn.==nextRegionFrom2)
         nexts= sidx[idx] 
         nexts= sidx[idx] 
         matrix[nexts,s]= matrix[nexts,s] .+ RegionService
         matrix[s,s]= matrix[s,s].- RegionService          
end
matrix
λidx

46-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0

In [292]:
print(matrix[:,6])

[0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.25, 0.25, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [293]:
matrix[length(sidx),:].= 1; #prep Ahat for matrix inversion

In [258]:
matrix

46×46 Matrix{Float64}:
 -1.0    0.0   1.25   0.0    0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   -1.0   0.0    1.25   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.25   0.0  -2.25   0.0    0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.25   0.5   0.0   -2.25   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.0   0.0    0.0   -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.0   0.0    0.0    0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.0   0.0    0.0    0.5      0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.0   0.0    0.0    0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.25   0.0   0.0    0.0    0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.5   0.0    0.0    0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.0   0.5    0.0    0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.0   0.0    0.0    0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0    0.0   0.0    0.0    0.0      0.0   0.0   0.0   0.0   0.0   0.0
  ⋮                               ⋱   ⋮  

In [259]:
inverse=inv(matrix); #invert Ahat matrix
#sum(inverse[:,length(sidx)]) #validate that sum of π equals 1

In [260]:
X=inverse[:,length(sidx)] #extract π from matrix

46-element Vector{Float64}:
 0.05147157273388092
 0.14145944093944973
 0.020588629093552302
 0.05657281293357943
 0.1414594409394494
 0.05166891469348775
 0.056594739817980216
 0.0206675658773951
 0.020588629093552337
 0.05659473981798022
 0.026765217821618035
 0.013228153084481713
 0.0026456306168963373
 ⋮
 0.003929718468249603
 0.00234850176352214
 0.00946246117258983
 0.002678435928167881
 0.026867835640613628
 0.005696986816980762
 0.009945508998093187
 0.004571941869970451
 0.001989101799618636
 0.0016402609173972733
 0.0004972754499046589
 0.019599357681897495

In [261]:
X=round.(X,digits=10); #round to 10 digits
sum(X)

0.9999999999000002

In [262]:
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
M=hcat(sidx,A,B,C,D,X)     #create readable matrix for state, sparti,spartj,spartk,spartl and pi

46×6 Matrix{Any}:
  1  1  1  1  1  0.0514716
  2  1  2  1  1  0.141459
  3  1  3  2  1  0.0205886
  4  1  4  1  2  0.0565728
  5  2  1  1  1  0.141459
  6  2  2  1  1  0.0516689
  7  2  3  2  1  0.0565947
  8  2  4  1  2  0.0206676
  9  3  1  2  1  0.0205886
 10  3  2  2  1  0.0565947
 11  3  3  3  1  0.0267652
 12  3  3  3  2  0.0132282
 13  3  3  3  3  0.00264563
  ⋮              ⋮
 35  4  3  3  4  0.00392972
 36  4  3  4  2  0.0023485
 37  4  3  4  3  0.00946246
 38  4  3  4  4  0.00267844
 39  4  4  1  3  0.0268678
 40  4  4  1  4  0.00569699
 41  4  4  2  3  0.00994551
 42  4  4  2  4  0.00457194
 43  4  4  3  3  0.0019891
 44  4  4  3  4  0.00164026
 45  4  4  4  3  0.000497275
 46  4  4  4  4  0.0195994

In [263]:
#Percentage of time with zero calls
Midx0= (spartk .+ spartl).==2
sum(M[Midx0.==1,6])

0.38605936920000006

In [264]:
#Percentage of time with 1 call
Midx1= (spartk .+ spartl).==6
sum(M[Midx1.==1,6])

0.0443921998

In [101]:
#Percentage of time with 2 calls 
Midx2= (spartj .+ spartk .+ spartl).==5
sum(M[Midx2.==1,6])

0.0310463079

In [102]:
#Percentage of time with 3 calls
Midx3= (spartj .+ spartk .+ spartl).==6
sum(M[Midx3.==1,6])

0.0542819014

In [103]:
#Percentage of time with 4 calls 
Midx4= (spartj .+ spartk .+ spartl).==7
sum(M[Midx4.==1,6])

0.07935388210000001

In [104]:
#Percentage of time with 5 calls 
Midx5= (spartj .+ spartk .+ spartl).==8
sum(M[Midx5.==1,6])

0.1268390758

In [105]:
#Percentage of time with 6 calls 
Midx5= (spartj .+ spartk .+ spartl).==9
sum(M[Midx5.==1,6])

0.17168729919999998

In [106]:
#Percentage of time with 7 calls 
Midx5= (spartj .+ spartk .+ spartl).==10
sum(M[Midx5.==1,6])

0.18873403219999998

In [110]:
#Percentage of time with max calls
MidxMax= (spartj .+ spartk.+ spartl).==(max1demand+max2demand+max3demand)
MaxInSys=sum(M[MidxMax.==1,6])

0.1273207536

In [108]:
#Midxw1Full= ((spartj .+ spartk).<=7).&(spartj.==max1demand)
#Midxw2Full= ((spartj .+ spartk).<=7).&(spartk.==max2demand);


In [282]:
#Define total throughput
#Throughput=((λ1+λ2)*MaxInSys+λ1*sum(M[Midxw2Full.==1,5])+λ2*sum(M[Midxw1Full.==1,5])) ###THIS IS WRONG
Throughput=(μ1Same)*sum(M[μ1idx.==1,6])+(μ1Diff)*sum(M[μ1idx.==2,6])+(μ2Same)*sum(M[μ2idx.==1,6])+(μ2Diff)*sum(M[μ2idx.==2,6]) ## This is somehow wrong- ask Dr. King


1.0544847926

In [297]:
Throughput=(λst)*sum(M[λidx.==1,6])*2

0.9739465728000001

In [283]:
sum(M[μ2idx.==2,6])+sum(M[μ2idx.==1,6])+sum(M[μ1idx.==2,6])+sum(M[μ1idx.==1,6])

0.9190337660000001

In [233]:
(μ2Diff)*sum(M[μ2idx.==2,6])

0.08380061535

In [16]:
#Reward Matrix

#Dwell Reward
Rmatrix= zeros(length(sidx),length(sidx))
Reward0=10
Reward1=5
Reward2=0
Reward3=-5
for s in 1:length(sidx)
    Call1 = spartj[s]
    Call2 = spartk[s]
    Z1Reward=0
    Z2Reward=0
    if     Call1==0
    Z1Reward=Reward0
    elseif Call1==1
    Z1Reward=Reward1
    elseif Call1==2
    Z1Reward=Reward2
    else   Call1==3
    Z1Reward=Reward3
    end
  
    if     Call2==0
    Z2Reward=Reward0
    elseif Call2==1
    Z2Reward=Reward1
    elseif Call2==2
    Z2Reward=Reward2
    else   Call2==3
    Z2Reward=Reward3
    end
    DwellReward=Z1Reward+Z2Reward
    
   Rmatrix[s,s]= DwellReward
end
Rmatrix

54×54 Matrix{Float64}:
 5.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  …   0.0   0.0    0.0    0.0    0.0
 0.0  5.0  0.0  0.0  0.0  0.0   0.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  5.0  0.0  0.0  0.0   0.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  …   0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0  -5.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0  -5.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  …   0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0      0.0   0.0    0.0    0.0    0.0
 0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0

In [17]:
#Transition Reward

RewardUp=-5
RewardDown=5

for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nexts=0
    Treward= 0
    
    
 #   Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2
        
        if Zoneflag <=3
            nextZoneflag = 3 #don't use double equal
        
        else Zoneflag == 4
            nextZoneflag = 4  #don't use double equal
        end
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2)
         nexts= sidx[idx]
         Rmatrix[nexts,s].=RewardUp
        
    end
      
    #Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1
        
        if Zoneflag !=3
            nextZoneflag = 4
        
        else Zoneflag == 3
                nextZoneflag = 3
        end
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2)
        nexts= sidx[idx] 
        Rmatrix[nexts,s].=RewardUp
    end
    
     #Service from Region to Region

    if Zoneflag == 3
        if Call1-1>=Call2 && Call1 > 2
            RegionService=μSame
            nextZoneflag=Zoneflag
            nextCall1=Call1-1
            nextCall2=Call2
            μidx=push!(μidx,1)
        
        elseif Call1-1>=Call2 && Call1==2
            RegionService=μSame
            nextZoneflag=Zoneflag-2
            nextCall1=Call1-1
            nextCall2=Call2
            μidx=push!(μidx,1)
            
        else
            RegionService=μDiff
            nextZoneflag= Zoneflag+1
            nextCall1= Call1-1
            nextCall2= Call2
            μidx=push!(μidx,2)
        end    
#         if Call1 == 2                     #Formulation for Finish Zone policy
#             if Call2 >=2
#             RegionService=μDiff
#             nextZoneflag=Zoneflag+1
#             nextCall1= Call1-1
#             nextCall2= Call2
#             else
#             RegionService=μSame
#             nextZoneflag=1
#             nextCall1= Call1-1
#             nextCall2= Call2
#             end
#             
#         else
#         RegionService=μSame
#         nextZoneflag= Zoneflag
#         nextCall1= Call1-1
#         nextCall2= Call2
#         end
        
    elseif Zoneflag == 4

        if Call2-1>=Call1 && Call2 > 2
            RegionService=μSame
            nextZoneflag=Zoneflag
            nextCall1=Call1
            nextCall2=Call2-1
            μidx=push!(μidx,1)
            
        elseif Call2-1>=Call1 && Call2==2
            RegionService=μSame
            nextZoneflag=Zoneflag-2
            nextCall1=Call1
            nextCall2=Call2-1
            μidx=push!(μidx,1)
            
                
        else
            RegionService=μDiff
            nextZoneflag= Zoneflag-1
            nextCall1= Call1
            nextCall2= Call2-1  
            μidx=push!(μidx,2)
        end
#         if Call2== 2                      #Formulation for Finish Zone Policy
#             if Call1 >=2
#             RegionService=μDiff
#             nextZoneflag=Zoneflag-1
#             nextCall1= Call1
#             nextCall2= Call2-1
#             else
#             RegionService=μSame
#             nextZoneflag=2
#             nextCall1= Call1
#             nextCall2= Call2-1     
#             end
        
#         else
#         RegionService=μSame
#         nextZoneflag= Zoneflag
#         nextCall1= Call1
#         nextCall2= Call2-1
        #         end 
    else Zoneflag <3
        
        nextZoneflag=Zoneflag
        nextCall1=Call1
        nextCall2=Call2

    end    
      #  println(RegionService)
      #  println(nextZoneflag)
      #  println(nextCall1)
      #  println(nextCall2)
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2)
        #println(idx)
        #println(sidx[idx])
         nexts= sidx[idx] 
       # println(nexts)
       # println("s:", string(s))
       # println(matrix[nexts,s])
    
    if Zoneflag >=3     
    
    Rmatrix[nexts,s].=RewardDown

    end
   
        
      
end
Rmatrix

54×54 Matrix{Float64}:
  5.0   0.0   0.0   0.0   0.0   0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   5.0   0.0   0.0   0.0   0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0   5.0   0.0   0.0   0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -5.0  -5.0  -5.0   0.0   0.0   0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -5.0  -5.0  -5.0   0.0   0.0   0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -5.0  -5.0  -5.0   0.0   0.0   0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0   0.0  -5.0  -5.0  -5.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0   0.0  -5.0  -5.0  -5.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0   0.0  -5.0  -5.0  -5.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -5.0  -5.0  -5.0   0.0   0.0   0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -5.0  -5.0  -5.0   0.0   0.0   0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -5.0  -5.0  -5.0   0.0   0.0   0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0   0.0   0.0  -5.0  -5.0  -5.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋮            

In [18]:
using LinearAlgebra
q=[]
for i in 1:length(sidx)
q= push!(q,dot(matrix[i,:],Rmatrix[i,:]))
end


In [19]:
q

54-element Vector{Any}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 15.0